In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import pickle
investment_data=pd.read_excel("investments.xlsx")

In [24]:
X = investment_data.drop(columns=['Investment'])
y = investment_data['Investment']

In [25]:
# Encode categorical variables
label_encoder = LabelEncoder()
X['Type'] = label_encoder.fit_transform(X['Type'])
X['Potential Return'] = label_encoder.fit_transform(X['Potential Return'])

In [26]:
X = X.values
y = y.values

In [27]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X, y)

DecisionTreeClassifier(random_state=42)

In [51]:
with open('investment_model.pkl', 'wb') as file:
    pickle.dump(model, file)

Model prediction: ['Savings Accounts']
